In [2]:
# importing necessary libraries
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [5]:

# getting data from internet
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wiki_page= requests.get(wiki_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wiki_page,'xml')
#print(soup.prettify())

# Processing to Extract raw table

In [7]:

# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            
            continue
            
           
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
            
            
    except:
        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        
        continue
        
        
        
        
        
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

# Processing to integrate Poscal Codes with neighbors greater than 1

In [9]:

unique_pc = set(Postcode)
print('num of unique Postal codes:', len(unique_pc))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_pc:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


# Create Dataframe

In [14]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
cols = df_toronto.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_toronto = df_toronto[cols]
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M6J,West Toronto,"Little Portugal, Trinity"
1,M9C,Etobicoke,Markland Wood
2,M1E,Scarborough,"Morningside, West Hill"
3,M4G,East York,Leaside
4,M2J,North York,Henry Farm
5,M5L,Downtown Toronto,Commerce Court
6,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market"
7,M1X,Scarborough,Upper Rouge
8,M6R,West Toronto,"Parkdale, Roncesvalles"
9,M1J,Scarborough,Scarborough Village


In [15]:
df_toronto.shape

(77, 3)